In [4]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
import math
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, roc_curve, auc

leaf_nodes=[]
child_tree = []
leaf_level_threshold=4



def binary_tree(result,final_cluster, count,tree_dic):

    first_cluster = []
    second_cluster = []
    if count==0:
        tree_dic[count]=result
    else:
        key=max(tree_dic)+1
        tree_dic[key] = result
    kmeans = KMeans(n_clusters=2, random_state=0).fit(result)
    score=0.1

    if score < 0.9 or count==0:
        for itr in range(len(result)):
            if final_cluster[itr] == 1:
                first_cluster.append(result[itr])
            else:
                second_cluster.append(result[itr])


        count = count + 1

        if len(first_cluster)> minimum_cluster_threshold :
           kmeans = KMeans(n_clusters=2, random_state=0).fit(first_cluster)

           binary_tree(first_cluster,kmeans.labels_,count,tree_dic)
        else:
            key = max(tree_dic) + 1
            tree_dic[key]=first_cluster
            leaf_nodes.append(key)

            if count <= leaf_level_threshold:
                child_tree.extend(first_cluster)

        if len(second_cluster) >minimum_cluster_threshold:
            kmeans = KMeans(n_clusters=2, random_state=0).fit(second_cluster)

            binary_tree(second_cluster,kmeans.labels_,count,tree_dic)
        else:
            key = max(tree_dic) + 1
            tree_dic[key] = second_cluster
            leaf_nodes.append(key)

            if count <= leaf_level_threshold:
                child_tree.extend(second_cluster)


    else:
        key = max(tree_dic)
        leaf_nodes.append(key)




def find_anomalies(lowest_leaf):
    ts = pd.read_csv(file_name, usecols=lambda column: column != 'Class').values

    y_pred = np.zeros(len(ts), dtype=int)

    lowest_leaf = np.array(lowest_leaf)

    for lowest_leaf_array in lowest_leaf:
        diffs = np.abs(ts - lowest_leaf_array)
        max_diffs = np.max(diffs, axis=1)
        outliers = max_diffs <= 1e-4
        y_pred[outliers] = 1

    return y_pred


if __name__ == '__main__':

    ts = pd.read_csv('4_breastw.csv')

    ts=np.array(ts)
    st_ts = ts.tolist()
    st_ts = sorted(st_ts)
    st_ts = np.array(st_ts)
    actual = st_ts[:, -1]
    ts = pd.read_csv('4_breastw.csv', usecols=lambda column: column != 'Class')
    ts=np.array(ts)
    size=len(ts)
    count = 0
    tree_dic = {}

    kmeans = KMeans(n_clusters=2, random_state=0).fit(ts)
    minimum_cluster_threshold = math.floor(0.2* size)
    final_cluster = kmeans.labels_
    binary_tree(ts, final_cluster, count, tree_dic)
    minkey = math.inf
    total_cluster_length = 0
    for key in tree_dic:
        if key in leaf_nodes:
            total_cluster_length = total_cluster_length + len(tree_dic[key])
            if key < minkey:
               minkey=key


    thresholds = [0.15, 0.1, 0.05,0.2]
    current_iteration = 0

    while len(child_tree) > 0.2 * size:
        count = 0
        leaf_level_threshold = 3
        tree_dic = {}
        leaf_nodes = []
        child_tree_copy = child_tree
        child_tree = []

        if current_iteration < len(thresholds):
            minimum_cluster_threshold = math.floor(thresholds[current_iteration] * len(child_tree_copy))
        else:
            # Use the last threshold value for all subsequent iterations
            minimum_cluster_threshold = math.floor(thresholds[-1] * len(child_tree_copy))

        kmeans = KMeans(n_clusters=2, random_state=0).fit(child_tree_copy)
        final_cluster = kmeans.labels_
        binary_tree(child_tree_copy, final_cluster, count, tree_dic)

        minkey = 0
        total_cluster_length = 0
        minarray = tree_dic[minkey]

        for key in tree_dic:
            if key in leaf_nodes:
                total_cluster_length = total_cluster_length + len(tree_dic[key])
            if len(tree_dic[key]) < len(minarray):
                minkey = key
                minarray = tree_dic[key]

        current_iteration += 1


    child_tree = np.vstack(child_tree)
    st_ts = ts.tolist()
    st_ts = sorted(st_ts)
    st_ts = np.array(st_ts)
    st_ft = child_tree.tolist()
    st_ft = sorted(st_ft)
    st_ft = np.array(st_ft)
    indices=[]
    count=0
    for i, element in enumerate(st_ts):
      if ((count<=(len(st_ft)-1)) and np.array_equal(st_ft[count], element)):
          indices.append(i)
          while i + 1 < len(st_ts) and np.array_equal(st_ts[i], st_ts[i + 1]):
              indices.append(i + 1)
              i += 1
          count += 1

    y_pred_test = np.zeros(len(ts), dtype=int)
    y_pred_test[indices]=1
    confusion_matrix_value = confusion_matrix(actual, y_pred_test)
    accuracy_value = accuracy_score(actual, y_pred_test)
    recall_mg = recall_score(actual, y_pred_test)
    precision_mg = precision_score(actual, y_pred_test)
    f1_score_mg = f1_score(actual, y_pred_test)

    fpr9, tpr9, thresholds = roc_curve(actual, y_pred_test)
    auc_roc_mg = auc(fpr9, tpr9)

    print()
    print(" Precision:{:.4f}".format(precision_mg))
    print(" Recall:{:.4f}".format(recall_mg))
    print(" F1-score:{:.4f}".format(f1_score_mg))
    print(" AUC-ROC:{:.4f}".format(auc_roc_mg))



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8


 Precision:0.8495
 Recall:0.3305
 F1-score:0.4759
 AUC-ROC:0.6495


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8